# ABC-XYZ анализ в Аптечной сети 

- дополнение к уже выполненому [ABC-анализу](https://github.com/Putinka1993/Simulate/blob/main/homework/ABC_analysis/ABC_solution.ipynb)

1. Агрегируем данные
   - по количеству товара в даты продаж
2. убираем пустые значения
3. собираем данные по товарам которые продаются не меньше чем 3 дня

In [ ]:
, group_table_XYZ as (
  select
    ds.dr_dat
    , ds.dr_ndrugs
    , sum(ds.dr_kol) as amount
  from drugs_simulative ds
  where ds.dr_ndrugs in (
    select ds2.dr_ndrugs
    from drugs_simulative ds2
    where ds2.dr_ndrugs != ''
    group by ds2.dr_ndrugs
    having count(distinct ds2.dr_dat) >= 3
  )
  group by ds.dr_dat, ds.dr_ndrugs
),

4. Подсчитываем коэффициент вариации
5. выставляем буквенные границы для определения XYZ анализа 

In [ ]:
XYZ_result as (
  select distinct
     dr_ndrugs
    , round(
      case when avg(amount) over(partition by dr_ndrugs) = 0 then null
        else stddev_pop(amount) over(partition by dr_ndrugs) / avg(amount) over(partition by dr_ndrugs)
      end, 3
    ) as COV
    , case
      	when round(stddev_pop(amount) over(partition by dr_ndrugs) / avg(amount) over(partition by dr_ndrugs), 3) <= 0.10 then 'X'
      	when round(stddev_pop(amount) over(partition by dr_ndrugs) / avg(amount) over(partition by dr_ndrugs), 3) <= 0.25 then 'Y'
      else 'Z'
    end as XYZ
  from group_table_XYZ
)

6. Объеденение таблицы и вывод ABC-XYZ 

In [ ]:
select
	a.dr_ndrugs
	, a.ABC_combined || '-' || coalesce(x.XYZ, 'N/A') as ABC_XYZ
from ABC_result a
left join XYZ_result x on a.dr_ndrugs = x.dr_ndrugs
order by
	a.ABC_combined
	, x.XYZ;

##  решение целиком 

In [ ]:
-- ABC-анализ
with
group_table as (
  select
    ds.dr_ndrugs
    , sum(ds.dr_kol) as amount 
    , sum(ds.dr_croz - ds.dr_sdisc) as profit_with_discount
    , sum(ds.dr_kol) / count(distinct dr_dat) * 30 as month_average
  from drugs_simulative ds
  where ds.dr_ndrugs != ''
  group by ds.dr_ndrugs
),
cumsum_table as (
  select
    dr_ndrugs
    , round(sum(amount) over(order by amount desc) / sum(amount) over() * 100.0, 2) as cumsum_count
    , round(sum(profit_with_discount) over(order by profit_with_discount desc) / sum(profit_with_discount) over() * 100.0, 2) as cumsum_profit_with_disc
    , round(sum(month_average) over(order by month_average desc) / sum(month_average) over() * 100.0, 2) as cumsum_month_average
    , case
      when sum(amount) over(order by amount desc) / sum(amount) over() * 100.0 <= 80 then 'A'
      when sum(amount) over(order by amount desc) / sum(amount) over() * 100.0 <= 95 then 'B'
      else 'C'
    end as ABC_count
    , case
      when sum(profit_with_discount) over(order by profit_with_discount desc) / sum(profit_with_discount) over() * 100.0 <= 80 then 'A'
      when sum(profit_with_discount) over(order by profit_with_discount desc) / sum(profit_with_discount) over() * 100.0 <= 95 then 'B'
      else 'C'
    end as ABC_profit_with_disc
    , case
      when sum(month_average) over(order by month_average desc) / sum(month_average) over() * 100.0 <= 80 then 'A'
      when sum(month_average) over(order by month_average desc) / sum(month_average) over() * 100.0 <= 95 then 'B'
      else 'C'
    end as ABC_month_average
  from group_table
),
ABC_result as (
  select
    dr_ndrugs
    , ABC_count || ABC_profit_with_disc || ABC_month_average as ABC_combined
  from cumsum_table
)
-- XYZ-анализ
, group_table_XYZ as (
  select
    ds.dr_dat
    , ds.dr_ndrugs
    , sum(ds.dr_kol) as amount
  from drugs_simulative ds
  where ds.dr_ndrugs in (
    select ds2.dr_ndrugs
    from drugs_simulative ds2
    where ds2.dr_ndrugs != ''
    group by ds2.dr_ndrugs
    having count(distinct ds2.dr_dat) >= 3
  )
  group by ds.dr_dat, ds.dr_ndrugs
),
XYZ_result as (
  select distinct
     dr_ndrugs
    , round(
      case when avg(amount) over(partition by dr_ndrugs) = 0 then null
        else stddev_pop(amount) over(partition by dr_ndrugs) / avg(amount) over(partition by dr_ndrugs)
      end, 3
    ) as COV
    , case
      	when round(stddev_pop(amount) over(partition by dr_ndrugs) / avg(amount) over(partition by dr_ndrugs), 3) <= 0.10 then 'X'
      	when round(stddev_pop(amount) over(partition by dr_ndrugs) / avg(amount) over(partition by dr_ndrugs), 3) <= 0.25 then 'Y'
      else 'Z'
    end as XYZ
  from group_table_XYZ
)
-- Объединение результатов ABC и XYZ
select
	a.dr_ndrugs
	, a.ABC_combined || '-' || coalesce(x.XYZ, 'N/A') as ABC_XYZ
from ABC_result a
left join XYZ_result x on a.dr_ndrugs = x.dr_ndrugs
order by
	a.ABC_combined
	, x.XYZ;